### Create Public Vector Database

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import CohereEmbeddings
import json
import random
from langchain.vectorstores import Chroma

# Set the game name 
game_name = 'Cyberpunk_2077'

# Load the file
with open(f'{game_name}/public_info.txt', errors='ignore') as f:
    public_info = f.read()

# Randomly select an API key
selected_key = json.load(open('apikeys.json', 'r'))['api_keys'][random.randint(
    0, len(json.load(open('apikeys.json', 'r'))['api_keys'])-1)]

# Initialise the embedding model
embeddings = CohereEmbeddings(cohere_api_key=selected_key)

# Create Text Splitter, Chunk size is number of characters in each item of the list 
# While chunk overlap is how many characters from the previous item should be repeated in the next item 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap  = 500,
    length_function = len,
    separators = ['\n\n\n', '\n\n', '\n', '.', ',', ' ']
)

# Split the text 
public_info_list = text_splitter.split_text(public_info)

# Path must be inside the game folder with the name public_vectordb
persist_directory = f'{game_name}/public_vectordb'

# Create the Vector Database
vectordb = Chroma.from_texts(public_info_list, embeddings, persist_directory=persist_directory)
vectordb.persist()

### Test the Vector Database

In [4]:
# search for a topic and see if you get relevant information, k is for number of similar items to return
topic = "Johnny Silverhand"
docs = vectordb.similarity_search(topic, k = 3)
similar_story_string = "\n".join(doc.page_content for doc in docs)
print(similar_story_string)

Johnny Silverhand, born Robert John Linder,[3] was a famous influential rockerboy and the lead singer of the band Samurai before its breakup in 2008.[4] A military veteran who defined the rockerboy movement to what it is today, he was the most prominent figure that fought against the corrupted NUSA government and megacorporations, often being described as a terrorist. Despite being charismatic and quite the charmer Johnny is also known to be irrational, impulsive, and a manipulator.[5] Silverhand got his nickname from a cybernetic implant left arm he had installed after losing it in the Second Central American War. One of the early (and most probable) victims of cyberpsychosis, he then became very temperamental and "on the edge". He was described as driven by "dedication and ambitions, but at the end of the day he doesn't care much for the people around him as long as they are used to accomplish his goals" in one interview.[6] Eventually, Johnny was killed by Adam Smasher during the Ni

### To add text to existing Public vector database 
#### (Optional, only if you want to add something that was not previously there in the above text file)

In [ ]:
# from langchain.embeddings import CohereEmbeddings
# import json
# import random
# from langchain.vectorstores import Chroma

#selected_key = json.load(open('apikeys.json', 'r'))['api_keys'][random.randint(
#    0, len(json.load(open('apikeys.json', 'r'))['api_keys'])-1)]
#embeddings = CohereEmbeddings(cohere_api_key=selected_key)
#persist_directory = 'Cyberpunk_2077/public_vectordb'
#vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
#vectordb.add_texts(["pineapples are yummy", "tomatoes are evil"])
#vectordb.persist()

#### To delete vectordb entirely, delete the public_vectordb folder inside the game folder

In [ ]:
import shutil
# path to public_vectordb inside your game folder
folder_path = 'Some_game/public_vectordb'
shutil.rmtree(folder_path)